# Reconstitution INPI

## Recreation du fichier INPI 

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from inpi_insee import preparation_data

%load_ext autoreload
%autoreload 2

In [ ]:
path = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching' \
r'\Notebooks_matching\programme_matching\data\output'

list_issue = []
df_matched = pd.DataFrame()
for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in tqdm(files):
        if name.endswith((".gz")):
            path_gz = '{}\{}'.format(root, name)
            df_ = pd.read_csv(path_gz, compression = 'gzip',low_memory = False)
            #if df_.loc[lambda x: x['siren'].isin([752085324])].empty == False:
                #print(name)
            df_matched = df_matched.append(
            df_
            )

In [ ]:
path_inpi = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching' \
r'\Notebooks_matching\programme_matching\data\input\INPI\inpi_etb_stock_0.gz'
df_inpi = pd.read_csv(path_inpi,compression = 'gzip',low_memory = False)

In [ ]:
df_inpi.shape[0] - df_matched.shape[0]

In [ ]:
#for i in list(df_matched):
#    if i not in list(df_inpi):
#        print(i)

In [ ]:
df_matched_full = df_inpi.merge(df_matched[
    ['index','siret','origin','count_initial_insee', 'count_siren_siret',
     'test_address_libelle','test_address_complement','test_join_address',
     'test_date','test_1','test_siege','test_voie','test_numero',
     'count_duplicates_final','count_duplicates_','test'
     ]
], on='index', how='inner')

In [ ]:
df_matched_full.shape

## Génération Detail sequence


In [ ]:
from inpi_insee import siretisation
import matplotlib.pyplot as plt

In [ ]:
siretisation.crate_graph_report_test(df_matched_full)

In [ ]:
siretisation.create_graph_report(df= df_matched_full)

## Rapport sur les logs

In [ ]:
import glob, os, json
import pandas as pd
data = []
#os.chdir(r"data\logs\")
for file in glob.glob(r"data\logs\*.json"):
    with open(file) as f:
        for line in f:
            data.append(json.loads(line))

In [ ]:
logs = pd.json_normalize(data)
logs

Nombre de lignes sirétisées

In [ ]:
logs[['total_match_rows_current']].sum()

Pourcentage de lignes sirétisées

In [ ]:
logs[['perc_total_match_rows_initial']].sum()

Nombre de lignes ayant été trouvé à l'INSEE

In [ ]:
logs[['perc_total_match_siren_initial']].sum()

Graphique sur la séquence ayant aidée à la sirétisation

In [ ]:
logs[['perc_total_match_rows_initial',
      'perc_total_match_siren_initial']].plot.bar(stacked=False)

Graphique sur la séquence avec le nombre de lignes sirétisées

In [ ]:
logs[['total_match_rows_current']].plot.bar(stacked=False)

Nombre de SIREN et index non sirétisés

In [ ]:
logs[["df_duplication.df_sp_index.nb_index",
      'df_duplication.df_sp_index.unique_siren']].plot.bar(stacked=False)

### Test generation SIREN aléatoire

Utile pour faire des vérifications dans l'App

In [ ]:
df_matched_full.loc[np.random.randint(low= 1, high= 7000000, size=1)[0]]

# Save SQL

In [ ]:
param = {
    'communes_insee': r'data\input\communes_france.csv',
    'upper_word':r'data\input\upper_stop.csv',
     "voie":r'data\input\voie.csv',
    'insee': r"data\RawData\INSEE\Stock\ETS\StockEtablissement_utf8.csv",
    'inpi_etb': r"data\RawData\INPI\Stock\initial_ETS.gz",
    'date_end':"2018-01-01"
}
prep_data = preparation_data.preparation(param)

In [ ]:
reindex = [
    "Code Greffe", "Nom_Greffe","Numero_Gestion","RCS_Registre","Date_Greffe",
    "Libelle_Evt","ID_Etablissement","siren", "siret","Nom_Commercial",
    "Enseigne","Date_Début_Activité","Domiciliataire_Nom",
    "Domiciliataire_Siren","count_initial_inpi","Domiciliataire_Greffe",
    "Domiciliataire_Complément","Type","Siège_PM","Activité","Origine_Fonds",
    "Origine_Fonds_Info","Type_Exploitation","Pays","Ville","ncc","Code_Postal",
 "Code_Commune","Adresse_Ligne1","Adresse_Ligne2","Adresse_Ligne3","Adress_new",
 "Adresse_new_clean_reg","possibilite","INSEE","digit_inpi","list_digit_inpi",
 "len_digit_address_inpi","Siege_Domicile_Représentant","Activité_Ambulante",
 "Activité_Saisonnière","Activité_Non_Sédentaire","origin","count_initial_insee",
 "count_siren_siret","test_address_libelle","test_address_complement",
 "test_join_address","test_date","test_1","test_siege","test_voie","test_numero",
 "count_duplicates_final","count_duplicates_","test"
]

In [ ]:
print('Creating SQL database')
query = "CREATE TABLE INPI_MATCHED ( \
Code Greffe, Nom_Greffe,Numero_Gestion,RCS_Registre,Date_Greffe,Libelle_Evt, \
 ID_Etablissement,siren, siret,Nom_Commercial,Enseigne,Date_Début_Activité,\
 Domiciliataire_Nom,Domiciliataire_Siren,count_initial_inpi,\
 Domiciliataire_Greffe,Domiciliataire_Complément,Type,Siège_PM,Activité,\
 Origine_Fonds,Origine_Fonds_Info,Type_Exploitation,Pays,Ville,ncc,Code_Postal,\
 Code_Commune,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,Adress_new,\
 Adresse_new_clean_reg,possibilite,INSEE,digit_inpi,list_digit_inpi,\
 len_digit_address_inpi,Siege_Domicile_Représentant,Activité_Ambulante,\
 Activité_Saisonnière,Activité_Non_Sédentaire,origin,count_initial_insee,\
 count_siren_siret,test_address_libelle,test_address_complement,\
 test_join_address,test_date,test_1,test_siege,test_voie,test_numero,\
 count_duplicates_final,count_duplicates_,test)"

prep_data.save_sql(
    df = df_matched_full.drop(columns = 'index').reindex(columns = reindex),
    db = r'App\SQL\inpi_matched.db',
    table = 'INPI_MATCHED',
    query =query)

In [ ]:
df_matched_full.to_csv('data\\inpi_etb_test_{}.gz'.format(
    df_matched_full.shape[0]),
                       compression='gzip', index = False) 